# Wards, Streets, Buildings and Deaths

## Download London wards

In [ ]:
import urllib.request as ur, os

download_link='https://data.london.gov.uk/download/statistical-gis-boundary-files-london/08d31995-dd27-423c-a987-57fe8e952990/London-wards-2018.zip'
target_file='data/londonwards.zip'

if not os.path.exists(target_file):
    ur.urlretrieve(download_link, target_file)

In [ ]:
!rm -fvR data/London-wards-2018_ESRI
!rm -fvR data/London-wards-2018_Mapinfo
!cd data && unzip londonwards.zip

## Start here if you have London wards already

In [ ]:
import geopandas as gpd

In [ ]:
london_gdf = gpd.read_file('data/London-wards-2018_ESRI/London_Ward_CityMerged.shp')

In [ ]:
london_gdf.plot()

In [ ]:
london_gdf.crs

In [ ]:
london_27700_gdf = london_gdf.copy()

In [ ]:
london_27700_gdf

In [ ]:
london_4326_gdf = london_gdf.to_crs('epsg:4326').copy()

In [ ]:
london_polygon = london_4326_gdf.geometry.unary_union

london_polygon

In [ ]:
wards = ['West End','Knightsbridge and Belgravia','Hyde Park','Bryanston and Dorset Square',\
        'Marylebone High Street','Bloomsbury','Holborn and Covent Garden','St. James\'s']
filter_polygon = london_4326_gdf[london_4326_gdf['NAME'].isin(wards)].geometry.unary_union
map_coords = (filter_polygon.centroid.y, filter_polygon.centroid.x)

In [ ]:
import folium

from folium import plugins

m = folium.Map(location=map_coords, zoom_start=13, control_scale=True)

for row in london_4326_gdf.itertuples():
    # Simplify the representation of the parishes
    #    so the polygons display easily
    sim_geo = gpd.GeoSeries(row.geometry).simplify(tolerance=0.000001)
    # convert the simplified geometry to GeoJSON
    geo_j = sim_geo.to_json()
    if row.NAME in wards:
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda x: {'color':'black','weight':3, 'fillColor': 'red','fillOpacity':0.2},
                               highlight_function=lambda x: {'color':'black','weight':3, 'fillColor': 'blue','fillOpacity':0.075})
    else:
        geo_j = folium.GeoJson(data=geo_j,
                               style_function=lambda x: {'color':'black','weight':1, 'fillColor': 'orange','fillOpacity':0.05},
                               highlight_function=lambda x: {'color':'black','weight':3, 'fillColor': 'blue','fillOpacity':0.075})
        
    folium.Popup('Ward: '+row.NAME+'<br/>'+'Area (ha): '+str(row.HECTARES)).add_to(geo_j)
    geo_j.add_to(m)
    
m

In [ ]:
london_4326_gdf.info()

In [ ]:
london_4326_gdf[london_4326_gdf['NAME'].isin(wards)].plot()

In [ ]:
london_4326_gdf[london_4326_gdf['NAME'].isin(wards)]

https://data.london.gov.uk/dataset/statistical-gis-boundary-files-london

https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.graph.graph_from_polygon

https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.plot.plot_graph

https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.utils.config

## Download Street Network

In [ ]:
import osmnx as ox, csv

with open('overpass-api.csv', mode='r') as infile:
    reader = csv.reader(infile)
    overpass_api = {rows[0]:rows[1] for rows in reader}

ox.config(
    log_console=False, 
    use_cache=True, 
    log_file=True,
    overpass_endpoint=overpass_api['main'],
    overpass_rate_limit=True,
    timeout=240
)

In [ ]:
download_graph = True

if download_graph:
    G = ox.graph_from_polygon(filter_polygon, network_type='all', simplify=False)
    G_proj = ox.project_graph(G)
    G_proj = ox.speed.add_edge_speeds(G_proj, precision=3)
    G_proj = ox.speed.add_edge_travel_times(G_proj, precision=3)
    G = ox.bearing.add_edge_bearings(G, precision=1)
    G = ox.distance.add_edge_lengths(G, precision=3)
    ox.save_graphml(G, 'data/g_unprojected.graphml')
    ox.save_graphml(G_proj, 'data/g_projected.graphml')
else:
    G = ox.load_graphml('data/g_unprojected.graphml')
    G_proj = ox.load_graphml('data/g_projected.graphml')

In [ ]:
G.graph['crs']

In [ ]:
G_proj.graph['crs']

UTM Zone 30: https://epsg.io/32630

In [ ]:
G_proj.graph['crs'] = 'epsg:32630'

In [ ]:
G_proj.graph['crs']

In [ ]:
ox.plot.plot_graph(G, figsize=(15,30), bgcolor='white', node_color='black')

## Download OSM building footprints

In [ ]:
%%time
if os.path.exists('data/footprints_gdf.gpkg'):
    footprints_gdf = gpd.read_file('data/footprints_gdf.gpkg')
else:
    footprints_gdf = ox.geometries.geometries_from_polygon(filter_polygon, tags={'building':True}).reset_index()

In [ ]:
footprints_gdf.plot(figsize=(15,30))

In [ ]:
cols_to_delete = footprints_gdf.columns[footprints_gdf.isnull().sum()/len(footprints_gdf) > .20]

In [ ]:
footprints_gdf.drop(cols_to_delete, axis = 1)

In [ ]:
footprints_gdf['building'].unique()

In [ ]:
footprints_gdf['centroid_x'] = footprints_gdf.geometry.centroid.x
footprints_gdf['centroid_y'] = footprints_gdf.geometry.centroid.y

In [ ]:
%%time
if 'nodes' in footprints_gdf.columns:
    footprints_gdf['nodes'] = footprints_gdf['nodes'].astype(str)
if 'ways' in footprints_gdf.columns:
    footprints_gdf['ways'] = footprints_gdf['ways'].astype(str)
    
footprints_gdf.to_file('data/footprints_gdf.gpkg', driver='GPKG')

In [ ]:
footprints_centroids_gdf = footprints_gdf[['osmid','building','centroid_x', 'centroid_y','geometry']].copy()
footprints_centroids_gdf['geometry'] = footprints_gdf['geometry'].to_crs('epsg:4326').centroid

footprints_centroids_gdf.to_file('data/footprints_centroids_gdf.gpkg', driver='GPKG')

In [ ]:
footprint_style = {'fillColor':'red','color':'black','weight':1,'opacity':0.5}
footprint_marker = {'radius':4}

In [ ]:
map1 = footprints_centroids_gdf.explore(tiles='CartoDB positron',style_kwds=footprint_style, marker_kwds=footprint_marker)
plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(map1)
map1

In [ ]:
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G, nodes=True, edges=True, node_geometry=True, fill_edge_geometry=True)

In [ ]:
node_style = {'fillColor':'blue','color':'black','weight':1,'opacity':0.5}
node_marker = {'radius':4}

In [ ]:
nodes_gdf.explore(m=map1, style_kwds=node_style, marker_kwds=node_marker)

In [ ]:
edges_gdf.explore(m=map1)

## Load Cholera deaths

In [ ]:
import pandas as pd

deaths_df = pd.read_csv('data/cholera_deaths.csv')
deaths_gdf = gpd.GeoDataFrame(deaths_df, geometry=gpd.points_from_xy(deaths_df.LON, deaths_df.LAT), crs='epsg:4326')
deaths_gdf.to_file('data/deaths_gdf.gpkg', driver='GPKG')

In [ ]:
deaths_style = {'fillColor':'black','color':'black','weight':2,'opacity':0.5, 'fillOpacity':0.8}
deaths_marker = {'radius':5}

In [ ]:
deaths_gdf.explore(m=map1, style_kwds=deaths_style, marker_kwds=deaths_marker)

https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html#geopandas-geodataframe-explore